# CHAPTER 7
# Data Wrangling : Clean, Transform, Merge, Reshape
<br/><br/><br/>

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt

np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))

from pandas import Series, DataFrame
import pandas
import pandas as pd

np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [2]:
%matplotlib inline

## 7.1 데이터 합치기 (Combining and Merging Data Sets)
### 7.1.1 데이터베이스 스타일로 DataFrame 합치기 (Database-style DataFrame Merges)

In [3]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [4]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [6]:
# many-to-one merge situation: 두 데이터프레임 중 하나만 key에 해당되는 value가 여러개일 때
# df1에서는 하나의 레이블(a나 b)에 여러 가지 값이 할당되어 있으나, df2에서는 하나의 레이블당 하나의 값만 할당되어 있는 경우
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [7]:
# 어떤 열을 기준으로 병합할지 지정하지 않으면, pandas.merge는 알아서 이름이 겹치는 열을 기준으로 병합한다. (위 셀의 경우)
# 이 셀에 있는 코드의 경우는 on='key'라는 옵션을 통해 key라는 열을 기준으로 병합하라고 지정한 경우.
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [8]:
#how='outer'옵션
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [9]:
#pd.concat()와 비교 - 밑에서 나갈 진도임
pd.concat([df1, df2], axis=1)

,data1,key,data2,key
0,0,b,0.0,a
1,1,b,1.0,b
2,2,a,2.0,d
3,3,c,NaN,NaN
4,4,a,NaN,NaN
5,5,a,NaN,NaN
6,6,b,NaN,NaN


In [10]:
#np.concatenate()와 비교 -> 데이터프레임으로 numpy의 함수 적용 불가.
np.concatenate([df1, df2], axis=1)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [11]:
#데이터프레임 새로 만들었음. 이번엔 column name이 다르다. (lkey vs. rkey)
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [12]:
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [13]:
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [14]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


#### 바로 위의 결과를 보면, df3에서는 c가 사라지고 df4에서는 d가 사라진다. 이는 pandas.merge는 디폴트로 'inner merge', 즉 두 데이터프레임의 intersection(교차점)인 key a와 b만을 결과에 보여주기 때문이다.
#### 만약 두 데이터프레임간 겹치지 않는 key도 결과에 나오게끔 하고 싶다면, how='outer' 옵션을 사용하면 된다.
#### pandas.merge에서 두 데이터프레임 병합 시 how= 옵션 뒤에 쓸 수 있는 것은 'left', 'right', 'out', 'inner' 네 가지가 있는데,
- how='left': 코드에서 왼쪽에 적힌 데이터프레임의 key 값에 대해서만 병합해서 보여준다. 
- how='right': 코드에서 오른쪽에 적힌 데이터프레임의 key 값에 대해서만 병합해서 보여준다. 
- how='outer': 두 데이터프레임에 있는 모든 key 값을 병합해서 보여준다.
- how='inner'(디폴트값): 두 데이터프레임에서 교차되는(두 데이터프레임에 다 들어 있는) key 값들만을 병합해서 보여준다.

In [15]:
# many-to-many merge situation: 두 데이터프레임 모두 한 key에 해당되는 value가 여러개일 때
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})

In [16]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [17]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [18]:
# key라는 column을 기준으로, df1에 있는 key 값들만을 병합한다.
# on='key'이므로 key라는 열을 기준으로 병합되었고, how=left이므로 df1에 있는 key 값들을 기준으로 병합되었음을 알 수 있다.
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [19]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [20]:
# To merge with multiple keys, pass a list of column names:
# 값을 기준으로 key를 연결할 때...? 하나의 값에 key가 여러개 붙어있는 형태
# 여러 개의 key값들을 병합하기 위해서는, 열 이름들의 목록을 넘겨야 한다.
# 아래 코드에서 알 수 있는 결과에서 생성될 열 이름들은 'key1', 'key2', lval', 'rval'
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': ['one', 'one', 'one', 'two'], 'rval': [4, 5, 6, 7]})
#left라는 데이터프레임의 key값들은 'foo', 'foo', 'bar'와 'one', 'two', 'one'
 
pd.merge(left, right, on=['key1', 'key2'], how='outer')
# 'key1'과 'key2'의 조합을 기준으로 병합을 실시함

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [21]:
# 'key1'을 기준으로 병합
# 'key2_x'는 left의 'key2' 값들이고 'key2_y'는 right의 'key2' 값들을 나타낸다.
# 여기서 left와 right의 열 이름이 겹치는데(overlapping),
# 이 때 결과로 출력되는 두 데이터프레임의 열 이름을 지정하는 옵션을 설정하지 않으면 디폴트로 '열이름_x'와 '열이름_y'로 출력된다.
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [22]:
# 두 데이터프레임에서 이름이 겹치는 열 이름 뒤에 붇는 접미사(suffix)를 지정해줄수 있는 옵션 suffixes=를 활용
# 아래 코드와 같이 접미사를 지정해줬으므로 'key2_x'와 'key2_y'로 나오지 않고 지정한 그대로 나온다.
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


##### pandas.merge의 추가적인 옵션들이 보고 싶다면 pdf 파일 181쪽 하단에 있는 Table 7-1 을 참고

### 7.1.2 색인 머지하기 (Merging on Index)
  데이터프레임에 있는 key 값의 인덱스를 병합의 기준으로 삼으려 할 때, <br />
'left_index=True' 나 'right_index=True' (혹은 둘 다) 옵션들을 지정하여<br />
병합에 사용되는 기준 key들을 인덱스 값들로 지정할 수 있다.

In [23]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [24]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [25]:
right1

,group_val
a,3.5
b,7.0


In [26]:
# right1에 저장해둔 index 값을 기준으로 left1과 right1을 병합했다.
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [27]:
# 마찬가지로 how='outer' 옵션을 통해 두 데이터프레임의 key값들의 교집합뿐만이 아닌 합집합을 결과로 나오게 할 수도 있다.
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [28]:
# 계층적 인덱스일 때는 어떻게 병합할까??

lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [29]:
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [30]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [31]:
# 일단 지역이름으로 병합하라고 명령 
pd.merge(lefth, righth, left_on='key1', right_index=True)

ValueError: len(left_on) must equal the number of levels in the index of "right"

In [ ]:
# 알아서 병합하지 못하고 에러가 뜬다.
# 왜? righth의 인덱스 계층이 2개이기 때문이다.
# 그래서 기준을 두 개 지정해주어야 한다.

In [32]:
# 즉, "리스트"로 여러 개의 기준 열을 지정해줘야 한다.
# lefth의 'key1'과 'key2'이라는 열과 righth의 인덱스를 기준으로 병합된다.
# how='outer'옵션이 없으므로 교집합 출력
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [33]:
# how='outer' 옵션까지 포함
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000.0,4.0,5.0
0,0.0,Ohio,2000.0,6.0,7.0
1,1.0,Ohio,2001.0,8.0,9.0
2,2.0,Ohio,2002.0,10.0,11.0
3,3.0,Nevada,2001.0,0.0,1.0
4,4.0,Nevada,2002.0,NaN,NaN
4,NaN,Nevada,2000.0,2.0,3.0


In [34]:
# 두 데이터프레임 모두의 인덱스를 사용하여 병합하는 것도 가능하다.
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'], 
                      columns=['Missouri', 'Alabama'])

In [35]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [36]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [37]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [38]:
# Q) how='outer' 옵션을 빼면?
pd.merge(left2, right2, left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [39]:
# 데이터프레임의 경우 인덱스로 병합할 때 'join'을 사용하여 더 간편하게 병합할 수 있다.
# 이는 같거나 비슷한 인덱스를 가졌지만 열 이름이 겹치지 않는 데이터프레임들을 결합할 때도 쓰인다.
# 바로 위 셀의 예제와 코드는 다르지만 동일한 결과가 나온다:
left2.join(right2, how='outer') #left2를 right2와 join함.

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [40]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [41]:
right1

,group_val
a,3.5
b,7.0


In [42]:
# 전해진 데이터프레임(여기서는 right1)의 인덱스를 호출한 데이터프레임(여기서는 left1)에 하나의 열처럼 결합시킬 수 있다.
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


### 7.1.3. 축따라 이어붙이기 (Concatenating Along an Axis)

Concatenate란?: to link or join together  
NumPy는 Numpy array들로 데이터를 묶는 'concatenate' 함수를 가지고 있다.

In [43]:
arr = np.arange(12).reshape((3, 4)) # 0부터 11까지의 정수를 3행 4열의 형태로 배열함

In [44]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [45]:
# numpy.concatenate를 통해 여러 개의 배열을 하나로 합칠 때
# 옵션 설정시 'axis=1'은 가로로 붙이기, 'axis=0'은 세로로 붙이기(=디폴트 값)
np.concatenate([arr, arr], axis=1) # arr와 arr을 가로로 이어 붙임!

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [46]:
# 세로로 배열을 이어붙이는 경우
np.concatenate([arr, arr], axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [47]:
# 인덱스가 서로 겹치지 않는 세 개의 Series를 지정

s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

# pandas.concat에 이들을 리스트 형태로 결합해서 돌려보면 모든 인덱스와 그에 해당하는 값들을 합쳐줌.
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [48]:
# pandas.concat도 numpy.concatenation과 마찬가지로 디폴트 옵션으로 'axis=0'이 지정된다. 이 경우 새 Series가 생성된다. 
pd.concat([s1, s2, s3], axis=0) # 'axis=0'은 디폴트 값이므로 바로 위 셀의 예제와 동일한 결과가 나온다.

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [49]:
# 만약 'axis=1' 옵션을 쓴다면, 그 결과는 데이터프레임으로 나올 것이다.(axis=1 is the columns)
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [50]:
s1

a    0
b    1
dtype: int64

In [51]:
s3

f    5
g    6
dtype: int64

In [52]:
# s1, s3와 겹치는 인덱스가 있게끔 적절히 조작하여 s4를 생성
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [53]:
# s1과 s4를 열로 이어붙여서 데이터프레임을 만들었다.
pd.concat([s1, s4], axis=1)  # 여기에 join='outer' 옵션을 포함하여도 결과는 동일

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [54]:
# 옵션을 join='inner'로 지정하면 교집합만을 결과로 보여준다.
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [55]:
# join_axes의 사용. 디폴트는 join_axes=None
# axis=1 일 경우 특정 DataFrame의 index를 그대로 이용하려면 입력 join_axes=[[대체할 인덱스들]] 입력
pd.concat([s1, s4], axis=1,
          join_axes=[['a', 'c', 'b', 'e']]) # 'a', 'c', 'b', 'e'로 concat을 실시

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


##### 여기서 잠깐! pandas.concat의 옵션 중 하나인 'join_axes'에 대한 추가 설명
- join_axes는 내가 결합하고 싶어하는 데이터프레임들의 인덱스들을 다른 인덱스들로 대체하고 싶을 때 쓴다.<br />
- 이 과정에서, 각 데이터프레임의 값들은 새 인덱스들에 assign되고, 해당 데이터프레임이 가지고 있던 인덱스들은 무시된다.<br />
- 또한, 만약 데이터프레임들 중 하나가 (어느 차원에서든) 새로 지정한 인덱스보다 길다면, 해당 데이터프레임에서 그 길이를 넘는 부분은 잘려나갈 것이다. 

In [56]:
s1

a    0
b    1
dtype: int64

In [57]:
s2

c    2
d    3
e    4
dtype: int64

In [58]:
s3

f    5
g    6
dtype: int64

In [61]:
# 이어 붙여진 형태를 파악하고 싶을 때, 계층적 인덱스(hierarchical index)를 지정할 수 있다.
# 'keys=[]' 옵션을 사용!
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [60]:
# Series들을 세로 방향(열 방향)으로(axis=1) 결합할 때,
##'keys=[]' 옵션으로 지정한 값들이 데이터프레임의 열 header들이 된다.

pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [62]:
# 이는 데이터프레임들을 결합할 때도 똑같이 적용된다.

df1 = pd.DataFrame(np.arange(6).reshape(3, 2),
                   index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2),
                   index=['a', 'c'],
                   olumns=['three', 'four'])

pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

TypeError: __init__() got an unexpected keyword argument 'olumns'

In [63]:
# 만약 리스트 대신 딕셔너리 형태로 결합할 객체들을 전달한다면,
##딕셔너리에 지정된 키 값들이 'keys=[]' 옵션에 자동으로 활용된다.

pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1      level2     
     one  two  data2  key
a    0.0  1.0    NaN  NaN
b    2.0  3.0    NaN  NaN
c    4.0  5.0    NaN  NaN
0    NaN  NaN    0.0    a
1    NaN  NaN    1.0    b
2    NaN  NaN    2.0    a
3    NaN  NaN    3.0    b
4    NaN  NaN    4.0    d

In [64]:
# Q) (결과 예상해보기)
pd.concat([df1, df2], axis=0, keys=['level1', 'level2'])

data2  key  one  two
level1 a    NaN  NaN  0.0  1.0
       b    NaN  NaN  2.0  3.0
       c    NaN  NaN  4.0  5.0
level2 0    0.0    a  NaN  NaN
       1    1.0    b  NaN  NaN
       2    2.0    a  NaN  NaN
       3    3.0    b  NaN  NaN
       4    4.0    d  NaN  NaN

In [65]:
# Q) pd.merge와 비교해보자. (결과 예상해보기)
pd.merge(df1, df2, left_index=True, right_index=True, how='outer')

,one,two,data2,key
a,0.0,1.0,NaN,NaN
b,2.0,3.0,NaN,NaN
c,4.0,5.0,NaN,NaN
0,NaN,NaN,0.0,a
1,NaN,NaN,1.0,b
2,NaN,NaN,2.0,a
3,NaN,NaN,3.0,b
4,NaN,NaN,4.0,d


In [66]:
# 추가적으로 계층적 인덱스(hierarchical index)들에 이름을 지정해주는 옵션도 있다.
# (교재 pdf파일 188쪽 하단의 표 7-2 참고!)
# names: Names for created hierarchical levels if keys and/or levels passed
pd.concat([df1, df2], axis=1,
          keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1      level2     
lower    one  two  data2  key
a        0.0  1.0    NaN  NaN
b        2.0  3.0    NaN  NaN
c        4.0  5.0    NaN  NaN
0        NaN  NaN    0.0    a
1        NaN  NaN    1.0    b
2        NaN  NaN    2.0    a
3        NaN  NaN    3.0    b
4        NaN  NaN    4.0    d

In [67]:
df1 = pd.DataFrame(np.random.randn(3, 4),
                   columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3),
                   columns=['b', 'd', 'a'])

In [68]:
df1

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221


In [69]:
df2

,b,d,a
0,0.274992,0.228913,1.352917
1,0.886429,-2.001637,-0.371843


In [70]:
# 행 인덱스가 별로 중요하지 않을 때, 'ignore_index=True' 옵션을 사용
# ignore_index=False: 기존 index 유지
# ignore_index=True: 기존 index 무시 -> 0부터 (행 수 -1)까지의 값을 행 인덱스로 사용함

pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,1.352917,0.274992,NaN,0.228913
4,-0.371843,0.886429,NaN,-2.001637


## 7.2 재형성과 피벗 (Reshaping and Pivot)
- 재형성 : 표 형식의 데이터를 재배치하는 다양한 기본 연산<br />
- stack : 데이터의 칼럼 -> (피벗 또는 회전) 로우 <br />
- unstack : 로우 -> 칼럼

In [71]:
#문자열이 담긴 배열을 로우와 칼럼의 색인으로 하는 작은 DataFrame
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index = pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns = pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [72]:
#stack 사용 : 칼럼 -> 로우
result = data.stack()

In [73]:
result #Series 객체 반환

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [74]:
#unstack 사용 : 로우 -> 칼럼
result.unstack() #DataFrame 반환

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [75]:
result.unstack().unstack()

number  state   
one     Ohio        0
        Colorado    3
two     Ohio        1
        Colorado    4
three   Ohio        2
        Colorado    5
dtype: int32

In [76]:
#레벨 이름이나 숫자를 전달해서 끄집어낼 단계 지정 가능
#(보통은 가장 안에 있는 레벨부터)
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [77]:
result.unstack(1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [78]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [79]:
#해당 레벨에 있는 모든 값이 하위 그룹에 속하지 않을 경우
s1 = Series([0,1,2,3], index = ['a','b','c','d'])
s2 = Series([4,5,6], index = ['c','d','e'])
data2 = pd.concat([s1, s2], keys = ['one','two'])

In [80]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [81]:
data2.unstack() #누락된 데이터 발생

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [82]:
#누락된 데이터가 발생한 칼럼을 다시 로우로 피벗하기
data2.unstack().stack() #누락된 데이터를 자동으로 걸러낸다

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [83]:
#누락된 데이터까지 포함시키고자 한다면
data2.unstack().stack(dropna=False) #dropna=False 옵션 추가

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [84]:
# Series인 'result'로 새로운 데이터프레임 생성
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [85]:
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))

In [86]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [87]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [88]:
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [89]:
#데이터베이스나 CSV 파일에 여러 개의 시계열 데이터를 저장하는 일반적인 방법
#시간 순으로 나열
data = pd.read_csv('macrodata.csv')

periods = pd.PeriodIndex(year = data.year, quarter = data.quarter, name = 'data')

data = DataFrame(data.to_records(),
                columns = pd.Index(['realgdp', 'inf', 'unemp'],
                                   name = 'item'),
                index = periods.to_timestamp(freq=None, how='end'))

ldata = data.stack().reset_index().rename(columns={0:'value'})

In [90]:
ldata[:10] #시계열 데이터가 시간 순으로 나열

,data,item,value
0,1959-03-31,realgdp,2710.35
1,1959-03-31,unemp,5.8
2,1959-06-30,realgdp,2778.8
3,1959-06-30,unemp,5.1
4,1959-09-30,realgdp,2775.49
5,1959-09-30,unemp,5.3
6,1959-12-31,realgdp,2785.2
7,1959-12-31,unemp,5.6
8,1960-03-31,realgdp,2847.7
9,1960-03-31,unemp,5.2


pivot 메서드 :
set_index를 사용해서 계층적 색인을 만들고
unstack로 형태를 변경하는 단축키 같은 메서드

In [91]:
#계층적 칼럼을 가지는 DataFrame을 얻기 위해 마지막 인자(name) 생략하기
pivoted = ldata.pivot('data', 'item')

In [92]:
pivoted[:5] #계층적 칼럼을 가지는 DataFrame

value      
item        realgdp unemp
data                     
1959-03-31  2710.35   5.8
1959-06-30   2778.8   5.1
1959-09-30  2775.49   5.3
1959-12-31   2785.2   5.6
1960-03-31   2847.7   5.2

In [93]:
pivoted['value'][:5] #'value2'를 제외한 'value' DataFrame만 출력

item,realgdp,unemp
data,,
1959-03-31,2710.35,5.8
1959-06-30,2778.8,5.1
1959-09-30,2775.49,5.3
1959-12-31,2785.2,5.6
1960-03-31,2847.7,5.2


In [94]:
#pivot()의 결과는 set_index()와 unstack()을 같이 쓴 결과와 같다.

un = ldata.set_index(['data', 'item'])
un[:7]

value
data       item            
1959-03-31 realgdp  2710.35
           unemp        5.8
1959-06-30 realgdp   2778.8
           unemp        5.1
1959-09-30 realgdp  2775.49
           unemp        5.3
1959-12-31 realgdp   2785.2

In [95]:
unstacked = ldata.set_index(['data', 'item']).unstack('item')
unstacked[:7]

value      
item        realgdp unemp
data                     
1959-03-31  2710.35   5.8
1959-06-30   2778.8   5.1
1959-09-30  2775.49   5.3
1959-12-31   2785.2   5.6
1960-03-31   2847.7   5.2
1960-06-30  2834.39   5.2
1960-09-30  2839.02   5.6

## 7.3 데이터 변형 (Data Transformation)

### 7.3.1 중복 제거하기 (Removing Duplicates)

- duplicated 메서드 : 각 로우가 중복인지 아닌지 알려주는 Boolean Series 객체 반환 <br />
- drop_duplicates 메서드 : duplicated 배열이 False인 DataFrame을 반환

In [96]:
data = DataFrame({'k1':['one']*3 + ['two']*4,
                 'k2':[1, 1, 2, 3, 3, 4, 4]})
data #중복된 로우 발견

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [97]:
#duplicated 메서드: 중복인지 아닌지 Boolean Series 객체 반환
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [98]:
#drop_duplicates 메서드: duplicated배열이 False인 DataFrame 반환
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [99]:
#중복을 찾아내기 위한 부분합 지정 가능
data['v1'] = range(7) #새로운 칼럼을 하나 추가
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [100]:
data.drop_duplicates(['k1']) #k1 칼럼에 기반에서 중복 걸러내기

,k1,k2,v1
0,one,1,0
3,two,3,3


In [101]:
data.drop_duplicates(['k1', 'k2']) #(옵션 없을 때)기본적으로 처음 발견된 값을 유지

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


In [102]:
#keep = 'last' 옵션 사용해서 마지막으로 발견된 값 반환하기
data.drop_duplicates(['k1', 'k2'], keep = 'last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


### 7.3.2 함수나 매핑 이용해 데이터 변형하기
DataFrame의 칼럼이나 Series, 배열 안의 값을 기반으로 데이터 형태 변형 가능 <br />
map 메서드 : 데이터의 요소별 변형 및 데이터를 다듬는 작업을 편리하게 수행

In [103]:
#가상 육류 데이터 형성
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami',
                           'honey ham', 'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [104]:
#해당 육류가 어떤 동물의 고기인기 알려줄 수 있는 칼럼 하나 추가
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}
meat_to_animal

{'bacon': 'pig',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon',
 'pastrami': 'cow',
 'pulled pork': 'pig'}

In [105]:
#map 메서드 : 사전류의 객체나 함수를 받는 메서드
#map 메서드로 객체 받기
data['animal'] = data['food'].map(str.lower).map(meat_to_animal) #모두 소문자로 변경
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [106]:
#map 메서드로 함수 받기
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.3.3 값 치환하기
fillna 메서드 : 누락된 값을 채우는 일반적인 값 치환 작업 <br />
map 메서드 : 한 객체 안에서 값의 부분집합을 변경하는 데 사용 <br/>
replace 메서드 : fillna 메서드, map 메서드와 비슷하지만 같은 작업에 대해서 좀 더 간단하고 유연한 방법을 제공

In [107]:
data = Series([1., -999., 2., -999., -1000.,3.]) #-999는 누락된 데이터를 나타내기 위한 값
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [108]:
#replace 메서드를 이용
#pandas에서 인식할 수 있는 NA 값으로 치환된 새로운 Series 생성
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [109]:
#여러 개의 값을 한 번에 치환
data.replace([-999, -1000], np.nan) #하나의 값 대신 치환하려는 값의 리스트를 넘김

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [110]:
#치환하려는 값마다 다른 값으로 치환
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [111]:
#2개의 리스트 대신 사전 이용
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64